This python script is used to get the qualifying and race position data from historical races for comparison in section 6, to see the impact of qualifying position on the final race outcome

In [2]:
import pandas as pd
import numpy as np
import formula1 as f1

In [3]:
master = pd.DataFrame()
# Loop setting the year
for n in range(1996, 2022):
    temp = pd.DataFrame()
    # Second loop selecting the race
    for i in range(1, 25):
        try:
            quali = f1.get_qualifying_result(n, i)
            race = f1.get_race_result(n, i)
            quali.rename(columns={'position': 'qualipos'}, inplace=True)
            quali['race'] = i
            quali = quali[['race', 'driverID', 'qualipos']]
            quali = quali.sort_values('driverID')
            quali = quali.set_index('driverID')
            race['race'] = i
            race.rename(columns={'position': 'racepos'}, inplace=True)
            race = race[['race', 'driverID', 'racepos']]
            race = race.sort_values('driverID')
            race = race.set_index('driverID')
            temp = temp.append(quali)
            temp['racepos'] = race['racepos']
        except IndexError:
            break
        # Error handling was used as number of races varied each year, so IndexErrors occurred if a value of a race greater than the total number of races that year was passed
        # This script breaks out of the second loop which selects the race as soon as there is an index error, so it obtains data for all races in a year (there are no years with more than 25 races)
    master = master.append(temp)


In [5]:
# Some drivers had NaN values for their finishing positon (didnt finish race). These values were removed below

master = master.reset_index()
dropdfs = master[master['racepos'].isnull()]
indexlist = dropdfs.index
indexlistarray = np.array(indexlist)
master.drop(indexlistarray, inplace=True)

In [17]:
master.to_csv('qualiracecorr.csv')